In [39]:
import pandas as pd
import numpy as np
from tensorflow.keras import Model,layers
import tensorflow as tf
np.random.seed(1337)  # for reproducibility
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding, LSTM, SpatialDropout1D
from keras.datasets import imdb
from keras.utils.np_utils import to_categorical
from sklearn.metrics import (precision_score, recall_score,
                             f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics
from sklearn.preprocessing import Normalizer

from keras import callbacks
from keras.callbacks import CSVLogger

import os

#traindata = pd.read_csv('preprocessed_train_multiclass(a)_new.csv', header=None)
#testdata = pd.read_csv('preprocessed_test_multiclass(a)_new.csv', header=None)
'''
X=pd.read_csv (r"E:\学习心得\data\cic-ids2018\02-03\ptrain1\ptrain2_sli121.csv", header=None )
T=pd.read_csv (r"E:\学习心得\data\cic-ids2018\02-03\ptrain1\ptrain2_sli11.csv", header=None )
C=pd.read_csv(r"E:\学习心得\data\cic-ids2018\02-03\ptrain1\ptrain2_sli1_label.csv",header=None)
Y=pd.read_csv(r"E:\学习心得\data\cic-ids2018\02-03\ptrain1\ptrain2_sli2_label.csv",header=None)
X = pd.read_csv(r"E:\学习心得\data\cic-ids2018\02-03\ptrain1\ptrain_1.csv", header=None)
Y = pd.read_csv(r"E:\学习心得\data\cic-ids2018\02-03\ptrain1\train_label.csv", header=None)

T = pd.read_csv(r"E:\学习心得\data\cic-ids2018\02-03\pva1\pva_1.csv", header=None)
C = pd.read_csv(r"E:\学习心得\data\cic-ids2018\02-03\pva1\pva_label.csv", header=None)'''

X = pd.read_csv(r"E:\学习心得\data\cic-ids2018\02-03\ptrain1\tr1\sptrain1_1_bert.csv", header=None)
Y = pd.read_csv(r"E:\学习心得\data\cic-ids2018\02-03\ptrain1\tr1\t1_2_label.csv", header=None)

T = pd.read_csv(r"E:\学习心得\data\cic-ids2018\02-03\ptrain1\tr1\sptrain1_2_bert.csv", header=None)
C = pd.read_csv(r"E:\学习心得\data\cic-ids2018\02-03\ptrain1\tr1\t2_2_label.csv", header=None)

X1 = X
T1 = T
X1.fillna(0,inplace=True)
X1.replace([np.inf,-np.inf],1,inplace=True)
T1.fillna(0,inplace=True)
T1.replace([np.inf,-np.inf],1,inplace=True)
#X1.value_counts('Label')
'''
for index in X1.columns:
    X1[index]=X1[index].fillna(0,inplace=True)
    X1[index]=X1[index].replace([np.inf,-np.inf],1,inplace=True)
'''

#scaler = Normalizer().fit(X1)
#trainX = scaler.transform(X1)
trainX =  np.array(X1)

#scaler = Normalizer().fit(T1)
#testT = scaler.transform(T1)
testT = np.array(T1)
y_train1 = np.array(Y)
y_test1 = np.array(C)

y_train = to_categorical(y_train1)
y_test = to_categorical(y_test1)

# reshape input to be [samples, time steps, features]
X_train = trainX
X_test = testT

print (trainX.shape )
#X_train = np.reshape(trainX, (trainX.shape[0],32,24))
#X_test = np.reshape(testT, (testT.shape[0],32,24))

# Add a channels dimension
X_train = X_train[..., tf.newaxis].astype("float32")
X_test = X_test[..., tf.newaxis].astype("float32")
batch_size = 50

print (X_train.shape )

train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train1)).shuffle(1000000).batch(50)

test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test1)).batch(50)


class MyModel(Model):
    #
    def __init__(self):
        #
        super(MyModel, self).__init__()
        '''
        self.lstm1=LSTM(50,input_shape=(1, 768), return_sequences= True )
        self.drop1=Dropout(0.01)
        self.lstm2=LSTM(50,return_sequences=True)
        self.drop2=Dropout(0.01)
        self.lstm3=LSTM(50, return_sequences=False)
        self.drop3=Dropout(0.01)
        '''
        self.flatten0=layers.Flatten(input_shape=(768,1))
        self.d0=layers.Dense(units=64,activation='relu',)
        self.cov1=layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 24, 1))
        self.pl1=layers.MaxPooling2D((2, 2))
       
        self.cov2=layers.Conv2D(64, (3, 3), activation='relu')
        self.pl2=layers.MaxPooling2D((2, 2))
        self.cov3=layers.Conv2D(64, (3, 3), activation='relu')
        self.pl3=layers.MaxPooling2D((2, 2))
        #self.flatten=layers.Flatten()
        #self.d0=layers.Dense(64, activation='relu')
        #self.drop1=Dropout(0.5)
        self.cov4=layers.Conv2D(64, (3, 3), activation='relu',padding='same')
        self.pl4=layers.MaxPooling2D((2, 2))
        self.cov5=layers.Conv2D(64, (3, 3), activation='relu',padding='same')
        self.flatten1=layers.Flatten()
        self.d1=layers.Dense(64, activation='relu')
        self.drop2=Dropout(0.5)
        self.d2=Dense(9)
        self.activation1=Activation('softmax')

    def call(self, x):
        #
        '''
       x = self.cov1(x)
        x = self.pl1(x)
        x = self.cov2(x)
        x = self.pl2(x)
        x = self.cov3(x)
       ''' 
       # x = self.pl3(x)
       # x = self.flatten(x)
       # x = self.d0(x)
       # x = self.drop1(x)
       # x = self.cov4(x)
       # x = self.pl4(x)
       # x = self.cov5(x)
       # x = self.flatten1(x)
        x = self.flatten0(x)
        x = self.d0(x)
        x = self.d1(x)
        x = self.drop2(x)
        x = self.d2(x)
        return self.activation1(x)

# Create an instance of the model
model = MyModel()

loss_object = tf.keras.losses.CategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')


@tf.function
def train_step(traind, labels):
    
    with tf.GradientTape() as tape:
        #
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
        predictions = model(traind, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

@tf.function
def test_step(images, labels):
    
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)
    
EPOCHS = 10

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
    )

#!mkdir -p saved_model
#model.save('saved_model/my_model_bert')
'''
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)
'''


(43983, 768)
(43983, 768, 1)
Epoch 1, Loss: 0.6749151945114136, Accuracy: 80.0400161743164, Test Loss: 2.1928868293762207, Test Accuracy: 73.81488037109375
Epoch 2, Loss: 0.4397260248661041, Accuracy: 85.35797882080078, Test Loss: 2.9420204162597656, Test Accuracy: 54.2459602355957
Epoch 3, Loss: 0.37971898913383484, Accuracy: 87.23825073242188, Test Loss: 3.1259591579437256, Test Accuracy: 56.69463348388672
Epoch 4, Loss: 0.35513001680374146, Accuracy: 88.0999526977539, Test Loss: 3.3113837242126465, Test Accuracy: 65.12970733642578
Epoch 5, Loss: 0.34126725792884827, Accuracy: 88.57968139648438, Test Loss: 3.5532243251800537, Test Accuracy: 68.11267852783203
Epoch 6, Loss: 0.33046087622642517, Accuracy: 88.81385803222656, Test Loss: 3.7527554035186768, Test Accuracy: 63.758724212646484
Epoch 7, Loss: 0.3160249888896942, Accuracy: 89.09806060791016, Test Loss: 4.076893329620361, Test Accuracy: 55.294090270996094
Epoch 8, Loss: 0.31089505553245544, Accuracy: 89.49139404296875, Test Los

"\nnew_model = tf.keras.models.load_model('saved_model/my_model')\n\n# Check its architecture\nnew_model.summary()\n# Evaluate the restored model\nloss, acc = new_model.evaluate(test_images, test_labels, verbose=2)\nprint('Restored model, accuracy: {:5.2f}%'.format(100 * acc))\n\nprint(new_model.predict(test_images).shape)\n"